In [1]:
import yaml

class Config:
    def __init__(self, config_file):
        self.config_file = config_file
        self.config = self.load_config()
        print("Loaded configuration:", self.config)

    def load_config(self):
        with open(self.config_file, 'r') as f:
            config = yaml.safe_load(f)
        return config

    def get_database_config(self):
        try:
            db_type = self.config['database']['type']
            print("Database type:", db_type)
        except KeyError:
            raise KeyError("No database type specified in configuration")
        
        try:
            db_config = self.config['database'][db_type.lower()]
            print("Original Database config:", db_config)
            db_config_with_type = db_config.copy()  # Create a copy to avoid modifying the original
            db_config_with_type['type'] = db_type  # Add the type to the config
            print("Modified Database config with type:", db_config_with_type)
            return db_config_with_type
        except KeyError:
            raise KeyError(f"No configuration found for database type '{db_type}'")

    def get_query(self, query_name):
        try:
            query = self.config['queries'][query_name]
            print("Query found:", query_name, "->", query)
            return query
        except KeyError:
            raise KeyError(f"No query found with name '{query_name}'")

# Test the get_database_config function
if __name__ == "__main__":
    config = Config("configs/config.yaml")
    db_config = config.get_database_config()
    print("Final Database config:", db_config)
    query = config.get_query('LOCAL_TO_DBBRUT_QUERY')
    print("Query:", query)


Loaded configuration: {'database': {'type': 'mysql', 'postgresql': {'user': 'postgres', 'password': 'your_password', 'host': 'localhost', 'port': 5432, 'dbname': 'dataraise_test'}, 'mysql': {'user': 'root', 'password': 'Sa*201299', 'host': '127.0.0.1', 'port': 3306, 'dbname': 'Sqops'}, 'sqlite': {'filepath': 'path/to/your/sqlite.db'}}, 'queries': {'LOCAL_TO_DBBRUT_QUERY': 'select distinct PROJECT_NAME, JOB_NAME, JOB_PATH, JOB_VERSION, Talend_Version from audit_jobs_delta where talend_version is not null and niveau is not null', 'LOCAL_TO_DBBRUT_FILTER_name_elementNode': "where aud_nameElementNode in ('DBNAME','TYPE','QUERY','TABLE','FILENAME','TEMPDIR','sql','query')", 'LOCAL_TO_DBBRUT_FILTER_columnName_metadata': "where aud_columnName not in ('errorCode','errorMessage')", 'LOCAL_TO_DBBRUT_QUERY_ROUTINES': 'select distinct PROJECT_NAME, ROUTINE_NAME, ROUTINE_PATH, ROUTINE_VERSION, Talend_Version from audit_routines where talend_version is not null', 'LOCAL_TO_DBBRUT_QUERY_CONTEXTGROUP'

In [2]:
import pymysql
import psycopg2
import sqlite3

class Database:
    def __init__(self, db_config):
        self.db_config = db_config
        self.connection = None
        self.cursor = None

    def connect(self):
        db_type = self.db_config['type']
        if db_type == 'mysql':
            self.connection = pymysql.connect(
                host=self.db_config['host'],
                port=self.db_config['port'],
                user=self.db_config['user'],
                password=self.db_config['password'],
                database=self.db_config['dbname']
            )
        elif db_type == 'postgresql':
            self.connection = psycopg2.connect(
                host=self.db_config['host'],
                port=self.db_config['port'],
                user=self.db_config['user'],
                password=self.db_config['password'],
                database=self.db_config['dbname']
            )
        elif db_type == 'sqlite':
            self.connection = sqlite3.connect(self.db_config['filepath'])
        else:
            raise ValueError(f"Unsupported database type: {db_type}")

        self.cursor = self.connection.cursor()

    def execute_query(self, query):
        if not self.connection:
            raise ValueError("Database connection is not established. Call connect() method first.")
        
        self.cursor.execute(query)
        return self.cursor.fetchall()

    def close(self):
        if self.cursor:
            self.cursor.close()
        if self.connection:
            self.connection.close()


In [2]:
from config import Config
from database import Database

# Example usage
if __name__ == "__main__":
    # Load configuration
    config = Config("configs/config.yaml")

    # Get selected database configuration and connect
    db_config = config.get_database_config()
    db = Database(db_config)
    db.connect()

    # Execute a query
    query = config.get_query('LOCAL_TO_DBBRUT_QUERY')
    results = db.execute_query(query)
    print("Query Results:", results)

    # Close the database connection
    db.close()


Loaded configuration: {'database': {'type': 'mysql', 'postgresql': {'user': 'postgres', 'password': 'your_password', 'host': 'localhost', 'port': 5432, 'dbname': 'dataraise_test'}, 'mysql': {'user': 'root', 'password': 'Sa*201299', 'host': '127.0.0.1', 'port': 3306, 'dbname': 'sqops_dataraise'}, 'sqlite': {'filepath': 'path/to/your/sqlite.db'}}, 'queries': {'LOCAL_TO_DBBRUT_QUERY': 'select distinct PROJECT_NAME, JOB_NAME, JOB_PATH, JOB_VERSION, Talend_Version from audit_jobs_delta where talend_version is not null and niveau is not null', 'LOCAL_TO_DBBRUT_FILTER_name_elementNode': "where aud_nameElementNode in ('DBNAME','TYPE','QUERY','TABLE','FILENAME','TEMPDIR','sql','query')", 'LOCAL_TO_DBBRUT_FILTER_columnName_metadata': "where aud_columnName not in ('errorCode','errorMessage')", 'LOCAL_TO_DBBRUT_QUERY_ROUTINES': 'select distinct PROJECT_NAME, ROUTINE_NAME, ROUTINE_PATH, ROUTINE_VERSION, Talend_Version from audit_routines where talend_version is not null', 'LOCAL_TO_DBBRUT_QUERY_CON

In [1]:
#C:/Users/sonia/Desktop/SQOPS_SERVER_PY/AUD_404_AGG_TAGGREGATE.py
from config import Config
from database import Database

def main():
    # Initialize the Config class with your configuration file
    config_file = "configs/config.yaml"
    config = Config(config_file)

    try:
        # Get MySQL database configuration and connect
        db_config = config.get_database_config()
        db = Database(db_config)

        # Connect to the database
        db.connect()

        # Test get_execution_date
        execution_date_query = config.get_query('queries', 'TRANSVERSE_QUERY_LASTEXECUTIONDATE')
        execution_date = db.get_execution_date(execution_date_query)
        print("Execution Date:", execution_date)

        # Test insert_data using table name 'aud_agg_aggregate'
        table_name = 'aud_agg_aggregate'
        insert_query = config.get_query("insert_queries",table_name)
        db.insert_data(insert_query, table_name)

    except Exception as e:
        print(f"An error occurred: {str(e)}")
    finally:
        # Close the database connection in the finally block to ensure it's always closed
        if db:
            db.close()

if __name__ == "__main__":
    main()


Loaded configuration: {'database': {'type': 'mysql', 'postgresql': {'user': 'postgres', 'password': 'your_password', 'host': 'localhost', 'port': 5432, 'dbname': 'dataraise_test'}, 'mysql': {'user': 'root', 'password': 'Sa*201299', 'host': '127.0.0.1', 'port': 3306, 'dbname': 'sqops_dataraise'}, 'sqlite': {'filepath': 'path/to/your/sqlite.db'}}, 'global_queries': {'LOCAL_TO_DBBRUT_QUERY': 'select distinct PROJECT_NAME, JOB_NAME, JOB_PATH, JOB_VERSION, Talend_Version from audit_jobs_delta where talend_version is not null and niveau is not null', 'LOCAL_TO_DBBRUT_FILTER_name_elementNode': "where aud_nameElementNode in ('DBNAME','TYPE','QUERY','TABLE','FILENAME','TEMPDIR','sql','query')", 'LOCAL_TO_DBBRUT_FILTER_columnName_metadata': "where aud_columnName not in ('errorCode','errorMessage')", 'LOCAL_TO_DBBRUT_QUERY_ROUTINES': 'select distinct PROJECT_NAME, ROUTINE_NAME, ROUTINE_PATH, ROUTINE_VERSION, Talend_Version from audit_routines where talend_version is not null', 'LOCAL_TO_DBBRUT_QU

In [1]:
from config import Config  # Assuming Config class is defined in config.py
from database import Database  # Assuming Database class is defined in database.py

def main():
    config_file = "configs/config.yaml"
    config = Config(config_file)

    jobs_paths = []  # Initialize the list to store job item paths

    try:
        # Retrieve JDBC parameters and create a Database instance
        jdbc_params = config.get_jdbc_parameters()
        print("JDBC Parameters:", jdbc_params)  # Print JDBC parameters to debug

        db = Database(jdbc_params)
        db.set_jdbc_parameters(jdbc_params)  # Set JDBC parameters if needed
        db.connect_JDBC()  # Test the JDBC connection

        # Step 1: Get the execution date
        execution_date_query = config.get_query('queries', 'TRANSVERSE_QUERY_LASTEXECUTIONDATE')
        print("Execution Date Query:", execution_date_query)  # Debug print
        execution_date = db.get_execution_date(execution_date_query)
        print("Execution Date:", execution_date)

        # Step 2: Execute LOCAL_TO_DBBRUT_QUERY
        local_to_dbbrut_query = config.get_query('queries', 'LOCAL_TO_DBBRUT_QUERY')
        print("Executing query:", local_to_dbbrut_query)  # Debug print
        local_to_dbbrut_query_results = db.execute_query(local_to_dbbrut_query)
        print("local_to_dbbrut_query_results", local_to_dbbrut_query_results)

        # Step 3: Delete the output from aud_elementvaluenode based on the query results
        for result in local_to_dbbrut_query_results:
            project_name, job_name, JOB_PATH, JOB_VERSION, Talend_Version = result  # Assuming result contains these fields in order
            print(f"Deleting records for PROJECT_NAME: {project_name}, JOB_NAME: {job_name}")  # Debug print
            db.delete_records(project_name, job_name)

            # Build the job item path
            job_item_path = f"{JOB_PATH}{job_name}{JOB_VERSION}.item"
            jobs_paths.append(job_item_path)

        # Step 4: Execute NOT_AUDITED_JOBS_QUERY
        not_audited_jobs_query = config.get_query('queries', 'NOT_AUDITED_JOBS_QUERY')
        print("Executing query:", not_audited_jobs_query)  # Debug print
        not_audited_jobs_query_results = db.execute_query(not_audited_jobs_query)
        print("not_audited_jobs_query_results", not_audited_jobs_query_results)

        # Step 5: Delete the output from aud_elementvaluenode based on the query results
        for result in not_audited_jobs_query_results:
            project_name, job_name = result
            print(f"Deleting records for PROJECT_NAME: {project_name}, JOB_NAME: {job_name}")  # Debug print
            db.delete_records(project_name, job_name)

        # Step 6: Output job item paths
        for path in jobs_paths:
            print(f"Job Item Path: {path}")

    except Exception as e:
        print(f"An error occurred: {str(e)}")

    finally:
        if db:
            db.close()  # Ensure the database connection is closed

if __name__ == "__main__":
    main()


Loaded configuration: {'database': {'type': 'mysql', 'postgresql': {'user': 'postgres', 'password': 'your_password', 'host': 'localhost', 'port': 5432, 'dbname': 'dataraise_test'}, 'mysql': {'user': 'root', 'password': 'Sa*201299', 'host': '127.0.0.1', 'port': 3306, 'dbname': 'sqops_dataraise'}, 'sqlite': {'filepath': 'path/to/your/sqlite.db'}}, 'queries': {'LOCAL_TO_DBBRUT_QUERY': 'select distinct PROJECT_NAME, JOB_NAME, JOB_PATH, JOB_VERSION, Talend_Version from audit_jobs_delta where talend_version is not null and niveau is not null', 'LOCAL_TO_DBBRUT_FILTER_name_elementNode': "where aud_nameElementNode in ('DBNAME','TYPE','QUERY','TABLE','FILENAME','TEMPDIR','sql','query')", 'LOCAL_TO_DBBRUT_FILTER_columnName_metadata': "where aud_columnName not in ('errorCode','errorMessage')", 'LOCAL_TO_DBBRUT_QUERY_ROUTINES': 'select distinct PROJECT_NAME, ROUTINE_NAME, ROUTINE_PATH, ROUTINE_VERSION, Talend_Version from audit_routines where talend_version is not null', 'LOCAL_TO_DBBRUT_QUERY_CON

In [13]:
import xml.etree.ElementTree as ET

# Load and parse the XML file
tree = ET.parse('C:/Users/sonia/Desktop/sqops/data/tFileInputXML_tFileOutputXML_0.1.txt')
root = tree.getroot()

# Extract and print data from `node` elements
for node in root.iter():
    if node.tag == 'node':
        # Extract and print the attributes of the `node`
        comp_data = {
            'componentName': node.get('componentName'),
            'componentVersion': node.get('componentVersion'),
            'offsetLabelX': node.get('offsetLabelX'),
            'offsetLabelY': node.get('offsetLabelY'),
            'posX': node.get('posX'),
            'posY': node.get('posY'),
        }
        print("Node Data:", comp_data)

        # Extract and print `elementParameter` data
        for elem_param in node.findall('.//elementParameter'):
            elem_data = {
                'field': elem_param.get('field'),
                'name': elem_param.get('name'),
                'show': elem_param.get('show'),
                'value': elem_param.get('value')
            }
            print("Element Parameter Data:", elem_data)

            # Extract and print `elementValue` data within `elementParameter`
            for elem_value in elem_param.findall('.//elementValue'):
                value_data = {
                    'elementRef': elem_value.get('elementRef'),
                    'value': elem_value.get('value')
                }
                print("Element Value Data:", value_data)


Node Data: {'componentName': 'tFileInputXML', 'componentVersion': '0.102', 'offsetLabelX': '0', 'offsetLabelY': '0', 'posX': '416', 'posY': '256'}
Element Parameter Data: {'field': 'TEXT', 'name': 'UNIQUE_NAME', 'show': 'false', 'value': 'tFileInputXML_1'}
Element Parameter Data: {'field': 'FILE', 'name': 'FILENAME', 'show': None, 'value': '"C:/Users/ineso/Desktop/Talend ESB/Studio/workspace/in.xml"'}
Element Parameter Data: {'field': 'TEXT', 'name': 'LOOP_QUERY', 'show': None, 'value': '"/bills/bill/line"'}
Element Parameter Data: {'field': 'TABLE', 'name': 'MAPPING', 'show': None, 'value': None}
Element Value Data: {'elementRef': 'SCHEMA_COLUMN', 'value': 'nom'}
Element Value Data: {'elementRef': 'QUERY', 'value': '"."'}
Element Value Data: {'elementRef': 'NODECHECK', 'value': 'false'}
Element Value Data: {'elementRef': 'SCHEMA_COLUMN', 'value': 'prenom'}
Element Value Data: {'elementRef': 'QUERY', 'value': '"."'}
Element Value Data: {'elementRef': 'NODECHECK', 'value': 'false'}
Elem